<a href="https://colab.research.google.com/github/aliash98/Search2Vec/blob/main/Search2Vec_SkipGram_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialisation

In [ ]:
import matplotlib.pyplot as plt
import itertools
from datetime import datetime
import pandas as pd
import numpy as np
import json
import re
import warnings
import gc 
from tqdm import tqdm, trange

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
% ls

gdrive/  sample_data/


In [ ]:
most_freq_q = pd.read_csv (r'./gdrive/MyDrive/Colab Notebooks/most_freq_queries_train.csv')

In [ ]:
most_freq_q.info()

In [ ]:
most_freq_q['raw_query'].value_counts().head(50)

In [ ]:
most_freq_c = pd.read_csv (r'./gdrive/MyDrive/Colab Notebooks/most_freq_clicks_train.csv')
most_freq_c.info()

In [ ]:
most_freq_p = pd.read_csv (r'./gdrive/MyDrive/Colab Notebooks/most_freq_products_train.csv')
most_freq_p.info()

# Construction of the Vocabulary

In [ ]:
# Constructing tokens
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0
# Importing queries to vocab
for query in most_freq_q.raw_query:
  if query not in vocab:
    vocab[query] = index
    index += 1


# *** Note: one of the below sections should be executed only!


# Importing products to vocab

# for product_id in most_freq_p._id:
#   if product_id not in vocab:
#     vocab[product_id] = index
#     index += 1

# Importing categories to vocab

for category in most_freq_p.category_name:
  temp_str = category + 'cat'
  if temp_str not in vocab:
    vocab[temp_str] = index
    index += 1

vocab_size = len(vocab)
print(vocab.keys())
print(vocab_size)

dict_keys(['<pad>', 'موبایل', 'Ps4', 'a52', 'لپ تاپ', 'گوشی', 'لب تاب', 'گوشی موبایل', 'A52', 'لپتاپ', 'پلی استیشن ۴', 'ps4', 'پلی استیشن 4', 'لبتاب', 'لب تاپ', 'پی اس فور', 'لپ تاب', 'گوشی شیایومی', 'تفنگ بادی', 'گوشی شیائومی', 'گوشی موبایل سامسونگ', 'ساعت هوشمند', 'گوشی سامسونگ', 'poco x3', 'شیایومی', 'شیامی', 'شیائومی', 'Poco x3', 'موبایل سامسونگ', 'شیاومی', 'گوشی شیامی', 'xiaomi', 'a21s', 'ایفون', 'ریش تراش', 'اسلایم', 'a10s', 'A21s', 'a32', 'شیرآلات', 'redmi note 10', 'شیر الات', 'هارد اکسترنال', 'A10s', 'شیرالات', 'آیفون', 'Redmi note 10', 'ردمی نوت ۱۰', 'A32', 'a10 s', 'iphone', 'ردمی نوت 10', 'هارد', 'هدفون', 'تلوزیون', 'مودم', 'اسپیکر', 'دلار', 'تلویزیون', 'اپل واچ', 'poco', 'a71', 'سامسونگ', 'a72', 'A72', 'poco m3', 'ردمی ۹', 'آیفون ۱۲', 'xbox', 'redmi 9', 'ایکس باکس', 'note 9 pro', 'note 8 pro', 'ایفون ۱۲', 'A71', 'iphone 12', 'redmi9', 'Iphone 12', 'Poco m3', 'پاور بانک', 'نوکیا', 'هارد اینترنال', 'note 8', 'کولرگازی', 'دریل', 'هندزفری بلوتوث', 's20 fe', 'هندزفری', 'ایفون ا

In [ ]:
# Exporting to a json file
json.dump( vocab, open( r'./gdrive/MyDrive/Colab Notebooks/vocab.json', 'w' ) )

## Inverting and Listing vocab

In [ ]:
inverse_vocab = {index: token for token, index in vocab.items()}
# print(inverse_vocab.keys())
# print(inverse_vocab[4])

# Extracting training sets

In [ ]:
most_freq_c.rename(columns={'search_log_id':'_id'}, inplace=True)
most_freq_c.info()

## Merging datasets

In [ ]:
# We should build targets (means queries), context (containing 1 + and 4 -), and it's labels
# For each query, we should retrieve it's clicks to get it's products
# Each of these query -> product denotes a positive sample
# Maybe we have to join our three data frames
# We don't need product data frame at all!
# So just joining Qs and Clicks
# A single Q can convert to a few entries
merged_q_c = pd.merge(most_freq_q, most_freq_c, how='inner', on=["_id"])
print(merged_q_c.info())
print(merged_q_c.head(5))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2236735 entries, 0 to 2236734
Data columns (total 6 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   _id              object
 1   raw_query        object
 2   datetime_x       object
 3   datetime_y       object
 4   rank             int64 
 5   base_product_id  object
dtypes: int64(1), object(5)
memory usage: 119.5+ MB
None
                        _id raw_query  ... rank base_product_id
0  6098383e97d4cd91f2a00f4d    موبایل  ...   25           mmmbh
1  60983843cfdd4491dedcbd97       Ps4  ...    1           mmmis
2  6098384f327e628bd4b38afe       a52  ...    4           mmyct
3  6098384f327e628bd4b38afe       a52  ...    4           mmyct
4  60983853dbcdf8a26ae98833    لپ تاپ  ...   56           mmnoo

[5 rows x 6 columns]


In [ ]:
# Just keeping the 'raw_query' and 'base_product_id' columns
merged_q_c = merged_q_c[['raw_query','base_product_id']]


# ***Note: these upcoming lines are just for categorical version
most_freq_p.rename(columns={'_id':'base_product_id'}, inplace=True)
# print(most_freq_p.info())
merged_q_c = pd.merge(merged_q_c, most_freq_p, how='inner', on=["base_product_id"])
merged_q_c = merged_q_c[['raw_query','category_name']]
gc.collect()

print(merged_q_c.info())
print(merged_q_c.head(5))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2236735 entries, 0 to 2236734
Data columns (total 2 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   raw_query      object
 1   category_name  object
dtypes: object(2)
memory usage: 51.2+ MB
None
  raw_query category_name
0    موبایل  گوشی سامسونگ
1      گوشی  گوشی سامسونگ
2      گوشی  گوشی سامسونگ
3      گوشی  گوشی سامسونگ
4      گوشی  گوشی سامسونگ


Checking categories of one significant query

In [ ]:
merged_q_c[merged_q_c['raw_query'] == 'کولر گازی'].category_name.value_counts().head(10)

کولر گازی                                   22389
سایر لوازم تهویه٬ سرمایش و گرمایش             149
فن پردازنده                                    51
محافظ، پریز، کلید و چندراهی برق                30
داکت اسپلیت                                    12
تجهیزات تعمیرگاهی خودرو                         6
شوفاژ و رادیاتور                                5
کتاب های صوتی و الکترونیکی                      5
کیف و کاور گوشی و تبلت                          5
سایر لوازم جانبی سیستم های صوتی و تصویری        4
Name: category_name, dtype: int64

## Putting into np arrays

In [ ]:
import random

default = np.zeros(5, dtype=int)
default[0] = 1
# print(default)
targets = []
contexts = []
labels = []
temp = np.array(5)
for ind in merged_q_c.index:
    targets.append(vocab[merged_q_c['raw_query'][ind]])
    temp = np.zeros(5, dtype=int)

    # ***Note: change here when you want to switch to product_id training mode

    temp[0] = vocab[merged_q_c['category_name'][ind] + 'cat']
    # getting 4 random negative samples
    for i in range(1, 5):
      temp[i] = random.randint(0, vocab_size - 1)
    contexts.append(temp)
    labels.append(default)

    # Including Reverse (X, Y)

    # targets.append(vocab[merged_q_c['category_name'][ind] + 'cat'])
    # temp = np.zeros(5, dtype=int)

    # # ***Note: change here when you want to switch to product_id training mode

    # temp[0] = vocab[merged_q_c['raw_query'][ind]]
    # # getting 4 random negative samples
    # for i in range(1, 5):
    #   temp[i] = random.randint(0, vocab_size - 1)
    # contexts.append(temp)
    # labels.append(default)


targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

gc.collect()
print(targets[:5])
print(contexts[:10,:5])
print(labels[:10,:5])


[1 5 5 5 5]
[[ 750  710  799  811 1122]
 [ 750 1039  894 1150 1113]
 [ 750  763 1011 1413  483]
 [ 750 1342   31 1479  306]
 [ 750  382  531  869  248]
 [ 750 1281  417 1249  760]
 [ 750  813  815    8  516]
 [ 750    4 1359  129  897]
 [ 750  209 1267 1365  792]
 [ 750 1220  365  372  242]]
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]]


In [ ]:
print(contexts.shape)
print(type(contexts))

(2236735, 5)
<class 'numpy.ndarray'>


# Setting up tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
AUTOTUNE = tf.data.AUTOTUNE

## Defining "dataset"

In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: (((1024,), (1024, 5)), (1024, 5)), types: ((tf.int64, tf.int64), tf.int64)>


In [ ]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset shapes: (((1024,), (1024, 5)), (1024, 5)), types: ((tf.int64, tf.int64), tf.int64)>


## Word2Vec class implementation

In [ ]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    
    # *** should we change right here?
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1,
                                       name="w2v_embedding2")

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

## Possible loss function alternative

In [ ]:
# Do ye need a change in our loss function?

# def custom_loss(x_logit, y_true):
#       return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

## Optimizer, Dims, loss

In [ ]:
embedding_dim = 300
num_ns = 4
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

## Main learning section

In [ ]:
gc.collect()
word2vec.fit(dataset, epochs=1, callbacks=[tensorboard_callback])

2184/2184 [==============================] - 47s 21ms/step - loss: 0.1753 - accuracy: 0.9877


In [ ]:
# #docs_infra: no_execute
# %load_ext tensorboard
# %tensorboard --logdir logs

## Extracting weights

In [ ]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
weights2 = word2vec.get_layer('w2v_embedding2').get_weights()[0]
average_weights = ( weights + weights2 ) / 2.0
print(average_weights.shape)
print(str(weights[0, 0]) + "  " + str(weights2[0, 0]) + "  " + str(average_weights[0, 0])) 

# vocab = vectorize_layer.get_vocabulary()

(1545, 300)
-0.03036716  -0.017832117  -0.024099639


In [ ]:
len(word2vec.get_layer('w2v_embedding').get_weights())
len(word2vec.get_layer('w2v_embedding2').get_weights())

1

Exporting weights to google drive 

In [ ]:
pd.DataFrame(average_weights).to_csv(r'./gdrive/MyDrive/Colab Notebooks/learned_wights.tsv', sep="\t", index=False, header=None)

An alternative for exporting it into files

In [ ]:
import io
# out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')
# out_v2 = io.open('vectors2.tsv', 'w', encoding='utf-8')
out_v_avg = io.open('vectors_avg.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  # vec = weights[index]
  # vec2 = weights2[index]
  # out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  # out_v2.write('\t'.join([str(x) for x in vec2]) + "\n")
  vec_avg = average_weights[index]
  out_v_avg.write('\t'.join([str(x) for x in vec_avg]) + "\n")
  out_m.write(word + "\n")
# out_v.close()
# out_v2.close()
out_v_avg.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
  # files.download('vectors.tsv')
  # files.download('vectors2.tsv')
  files.download('vectors_avg.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(most_freq_p['category_name'].value_counts().shape)
my_product = most_freq_p[most_freq_p['_id'] == 'mzkmd']
print(my_product['category_name'])
print(my_product['sellers'])

(756,)
36343    ماشین اصلاح و ریش‌ تراش
Name: category_name, dtype: object
36343    [{'name1': 'ریش تراش فیلیپس (هلند) 5000 S5672/...
Name: sellers, dtype: object


In [ ]:
print(average_weights[:5,:5])

[[-0.02409964  0.13826536 -0.18278532  0.01125851 -0.04670435]
 [-0.15826955  0.04051213 -0.08192925 -0.1244172  -0.06435126]
 [ 0.07748422  0.12065294 -0.06195807  0.07564399  0.03261204]
 [-0.02833261  0.10057643 -0.11756508 -0.0561972  -0.04967486]
 [ 0.1147184   0.04362245 -0.09093378 -0.07859519 -0.00983895]]
